In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-03 03:09:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.14MB/s    in 0.2s    

2022-01-03 03:09:02 (5.14 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("Q1").getOrCreate()
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.head()

Row(marketplace='US', customer_id='45610553', review_id='RMDCHWD0Y5OZ9', product_id='B00HH62VB6', product_parent='618218723', product_title='AGPtek® 10 Isolated Output 9V 12V 18V Guitar Pedal Board Power Supply Effect Pedals with Isolated Short Cricuit / Overcurrent Protection', product_category='Musical Instruments', star_rating='3', helpful_votes='0', total_votes='1', vine='N', verified_purchase='N', review_headline='Three Stars', review_body='Works very good, but induces ALOT of noise.', review_date='2015-08-31')

In [5]:
# Show first 10 records of a dataframe
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [6]:
#Count the number of records in a dataframe
df.count()

904765

In [7]:
#Transform the dataset to fit the review_id_table schema
review_id_table_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
#Change review_date column to date type
review_id_table_df=review_id_table_df.withColumn("review_date",review_id_table_df.review_date.cast("Date"))
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [8]:
#Print table schema
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# Transform the dataset to fit the products table schema with unique product ID
products_df=df.select(["product_id","product_title"]).dropDuplicates(['product_id'])
products_df.show(10)
products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0577088726|Sisyphus Redux fo...|
|0594478944|Nook Audio OE250 ...|
|0634010263|Hal Leonard Maná ...|
|0634029355|Hal Leonard Eric ...|
|0634056727|Hal Leonard My Fi...|
|0692218645|Ukulele Chord Cha...|
|0739050583|Alfred Billy Ward...|
|073906407X|Alfred Alfred's P...|
|0739075934|Alfred BILLY MART...|
|0739081381|Brandi Carlile Th...|
+----------+--------------------+
only showing top 10 rows



123328

In [10]:
#Transform the datasets to customers table with unique customer ID as "long" type and count of times this customer wrote reviews 
customers_df=df.select("customer_id").groupby("customer_id").count()
customers_df=customers_df.withColumn("customer_id",customers_df.customer_id.cast("Long")).withColumnRenamed("count", "customer_count")

customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9163460|             2|
|   50809283|             1|
|   13005724|             1|
|   51506042|             1|
|   48752158|            22|
|   12318800|             2|
|   26246534|             2|
|   47577568|             1|
|   15963400|            10|
|     134314|             1|
+-----------+--------------+
only showing top 10 rows



In [11]:
customers_df.printSchema()

root
 |-- customer_id: long (nullable = true)
 |-- customer_count: long (nullable = false)



In [12]:
#Transform the datasets to vine table
vine_table_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table_df=vine_table_df.withColumn("star_rating",vine_table_df.star_rating.cast("Long"))
vine_table_df=vine_table_df.withColumn("helpful_votes",vine_table_df.helpful_votes.cast("Long"))
vine_table_df=vine_table_df.withColumn("total_votes",vine_table_df.total_votes.cast("Long"))
vine_table_df.show(10)
vine_table_df.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|
| RZSL0BALIYUNU|          5|            0|          0|   N|
| RIZR67JKUDBI0|          3|            0|          1|   N|
|R27HL570VNL85F|          5|            0|          0|   N|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|
|R30SHYQXGG5EYC|          5|            0|          0|   N|
|R14YLXA56NP51I|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

root
 |-- review_id: string (nullable = true)
 |-- star_rating: long (nullable = true)
 |-- helpful_votes: long (nullable = true)
 |--

In [13]:
# Amazon RDS endpoint 

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<aws endpoint>:5432/mypostgresdb"
config = {"user":"postgres", 
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to review_id table in RDS

review_id_table_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)